In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import pytesseract

In [ ]:
patient_medic_events = pd.read_csv("patient_medicine_events_at_QA_by_label.txt", sep='\t')

In [ ]:
floors = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [ ]:
patient_medic_events['label'] = patient_medic_events['label'].str.replace('Wards ','')
patient_medic_events['label'] = patient_medic_events['label'].str.replace('Ward ','')
patient_medic_events['label'] = patient_medic_events['label'].str.replace(' Unit','')
patient_medic_events['label'] = patient_medic_events['label'].str.replace(' Centre','')
patient_medic_events['label'] = patient_medic_events['label'].str.replace('Level ','')
patient_medic_events['label'] = patient_medic_events['label'].str.replace(' Intensive Care','')
patient_medic_events['label'] = patient_medic_events['label'].str.replace(' Day Case','')
patient_medic_events['label'] = patient_medic_events['label'].str.replace('Maternity ','')
patient_medic_events['label'] = patient_medic_events['label'].str.replace('Renal Haemodialysis','Haemodialysis')
patient_medic_events.loc[patient_medic_events.label == 'Critical Care (E5)', 'label'] = 'E5'
patient_medic_events.loc[patient_medic_events.label == 'Renal', 'label'] = 'Renal Unit'
print(patient_medic_events)

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text_dict = {}
for floor in floors:
    text = pytesseract.image_to_data(Image.open('level_maps_'+floor.lower()+'.png'),output_type='data.frame')
    text = text.dropna()
    text = text[text.text != ' ']
    text = text[text.text != '  ']
    text = text[text.text != '   ']
    text = text[text.text != '    ']
    text = text[text.text != '     ']
    text_dict[floor] = text
print(text_dict)

In [ ]:
text_dict['A'].loc[text_dict['A'].text == 'A6', 'text'] = 'A5, A6'
text_dict['A']['text'] = text_dict['A']['text'].str.replace('Dischage','Discharge')
text_dict['B'].loc[text_dict['B'].text == 'Centre', 'text'] = 'Haematology & Oncology'
text_dict['B'].loc[text_dict['B'].text == 'B6', 'text'] = 'B5, B6 & B7'
text_dict['B'].loc[text_dict['B'].text == 'Eye', 'left'] = 1100
text_dict['C'].loc[text_dict['C'].text == 'Day', 'text'] = 'Acute Medical'
text_dict['C'].loc[text_dict['C'].text == 'Acute Medical', 'top'] = 354
text_dict['C'].loc[text_dict['C'].text == 'Ambulance', 'left'] = 700
text_dict['C'].loc[text_dict['C'].text == 'C6', 'text'] = 'C5, C6 & C7'
text_dict['C'].loc[text_dict['C'].text == 'Ambulance', 'text'] = 'C Outpatients'
text_dict['C'].loc[text_dict['C'].text == 'C Outpatients', 'left'] = 50
text_dict['C'].loc[text_dict['C'].text == 'Diabetes', 'text'] = 'Diabetes and Endocrinology'
text_dict['C'].loc[text_dict['C'].text == 'Blood', 'text'] = 'Emergency Department'
text_dict['C'].loc[text_dict['C'].text == 'Emergency Department', 'left'] = 1100
text_dict['C'].loc[text_dict['C'].text == 'Radiolo', 'text'] = 'Radiology'
text_dict['C'].loc[text_dict['C'].text == 'Rheumatology', 'text'] = 'Rheumatology Outpatients'
text_dict['D'].loc[text_dict['D'].text == 'D3', 'text'] = 'D1, D2, D3 & D4'
text_dict['D'].loc[text_dict['D'].text == 'Day', 'text'] = 'Day Surgery'
text_dict['D'].loc[text_dict['D'].text == 'Level', 'text'] = 'D Outpatients'
text_dict['D'].loc[text_dict['D'].text == 'Gynaecology', 'text'] = 'Gynaecology Outpatients'
text_dict['D'].loc[text_dict['D'].text == 'Neurophysiology', 'text'] = 'Neurophysiology Urology Outpatients'
text_dict['E'].loc[text_dict['E'].text == 'E5)', 'text'] = 'E5'
text_dict['E'].loc[text_dict['E'].text == 'E3', 'text'] = 'E1, E2, E3 & E4'
text_dict['E'].loc[text_dict['E'].text == 'E7', 'text'] = 'E6, E7 & E8'
text_dict['E'].loc[text_dict['E'].text == 'Theatre', 'text'] = 'Theatre Admissions'
text_dict['F'].loc[text_dict['F'].text == 'F6', 'text'] = 'F5, F6 & F7'
text_dict['F'].loc[text_dict['F'].text == 'F3', 'text'] = 'F1, F2, F3 & F4'
text_dict['F'].loc[text_dict['F'].text == 'Medical', 'text'] = 'Medical Physics'
text_dict['G'].loc[text_dict['G'].text == 'Unit', 'text'] = 'Renal Unit'
text_dict['G'].loc[text_dict['G'].text == 'G8,', 'text'] = 'G5'
text_dict['G'].loc[text_dict['G'].text == 'G5', 'top'] = 750
text_dict['G'].loc[text_dict['G'].text == 'G5', 'left'] = 700
text_dict['G'].loc[text_dict['G'].text == 'G2,', 'text'] = 'G1, G2, G3 & G4'
text_dict['G'].loc[text_dict['G'].text == 'G7', 'text'] = 'G8, G7 & G6'
text_dict['G'].loc[text_dict['G'].text == 'G10', 'text'] = 'G10 & G9'
text_dict['G'].loc[text_dict['G'].text == 'Outpatients', 'text'] = 'Renal Outpatients'

In [ ]:
merged_dict = {}
for floor in floors:
    merged_dict[floor] = pd.merge(left=patient_medic_events,right=text_dict[floor],left_on='label',right_on='text')
print(merged_dict)

In [ ]:
cm = plt.cm.get_cmap('RdYlGn_r')
vmin = patient_medic_events['events'].min()
vmax = patient_medic_events['events'].max()
area_multiplication = 6

In [ ]:
for floor in floors:
    plt.figure()
    img=mpimg.imread('level_maps_'+floor.lower()+'.png')
    plt.figure(figsize=(60,6))
    imgplot = plt.imshow(img)
    fig = plt.gca()
    x = merged_dict[floor].left.values + merged_dict[floor].width.values/2
    y = merged_dict[floor].top.values + merged_dict[floor].height.values/2
    size = merged_dict[floor].events.values*area_multiplication
    color = merged_dict[floor].events.values
    plt.scatter(x,y,s=size,c=color,vmin=vmin,vmax=vmax,cmap=cm,alpha=0.5)
    plt.colorbar(label='# of medicine patient safety events from 31/3/16-2/10/19')
    fig.get_xaxis().set_visible(False)
    fig.get_yaxis().set_visible(False)
    plt.savefig('floor_'+floor.lower()+'_by_label.png')